In [1]:
import numpy as np
import matplotlib as plot
import pickle
import tarfile
import os
import requests
from collections import namedtuple
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
from IPython.utils.io import capture_output
InteractiveShell.ast_node_interactivity = "all"

In [7]:
seed = 1

# Intro
Simple notebook showing the steps to implement a NN from scratch. This notebook consists of two big parts, implementing one-layer nn and then extending it to m-layers nn. For this notebook, we are going to implement a one linear layer neural network with no activation for simplicity.  The task is image classifciation on cifar10 dataset, and the hope is to get a performance better than random guessing which given the 10 classes, should be $> 1/10$. This would indicate that it at least learnt something.

# Purpose 
There are lots of tutorials online, that are better than this notebook I'm constructing, like Karpathy's famous [micrograd series](https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ), but I wanted to take a more **analytical approach** as per my previous labs in a DL course I took years ago, showing how to go from a computational graph to deriving the math for the gradients to implementing them in a backprop. The hope is to show that this is a very mechanical process, aside from deriving some of the tricky gradients, there's a very clear pattern on how to work with this. What I had in mind should algorithmitcally be very straightforward, however the math for some of the gradients aren't trivial, and sometimes requires drawing the matrices to understand what's really happening. That's the only part I didn't find a simpler way to deal with, which I disliked, although I do think there are easier ways to work with them, something something about proper matrix calculus techniques. Admittedly, Karpathy's video is easier to understand and shorter code, since it's an implementation of automatic differentiation, whereas this implementation is purely analytical and not automatic. Regardless, it can serve as an alternative approach, to give a different perspective as it doesn't require implementing an automatic differentiation engine.

# Prerequisites
Basics of chain rule and familiarity with matrix calculus and working with matrices in general.

# Preprocess data
Originally 10000 x 32 x 32 x 3, but need to transpose it to get the same shape as in the formulas for the gradients in the derivations. There are 5 files + 1 test file and each file has 10k images. The test file has 10k as well.

* We will extract the first 10k for training and next 10k for validation. Also need to be in $d \times n$ shape, where d is the features and n the number of samples. The test data will be from the test_batch file.
* Labels should be one-hot encoding and in shape $c \times n$, where c is the classes and n the number of samples.
* Need to normalize data, subtract all data by training mean and divide by training std to turn it into zero mean unit variance data. This seems to be a [common practise](https://stats.stackexchange.com/a/202290), since the nn will be trained on the training data and therefore use those statistics. To make those statistics compatible we there need to transform the validation and test data the same way.

### Check where current folder is
We should access the cifar-10-python-tar.gz file in the data folder and extract it in the same folder. Adjust the paths accordingly if they are not correct. If the current folder path is chapter6, then it's correct.

In [ ]:
current_path = os.getcwd()
print("Current folder path:", current_path) 

In [2]:
DATA_PATH = os.path.join("data")
CIFAR_ROOT_PATH = os.path.join(DATA_PATH, "cifar-10-batches-py")
TEST_DATA_BATCH_PATH = os.path.join(CIFAR_ROOT_PATH, "test_batch")
CIFAR_TAR_PATH = os.path.join(DATA_PATH, "cifar-10-python.tar.gz")

In [6]:
if not os.path.exists(CIFAR_TAR_PATH):
    # Download cifar10, might take some time depending on internet speed, can take anywhere from few secs to a few minutes. Took me around 2mins.
    try:
        with requests.get("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", stream=True, timeout=300) as response:
            response.raise_for_status()  # Raise an error for bad status codes
            with open(CIFAR_TAR_PATH, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    with capture_output():
                        file.write(chunk)
        print("Download completed successfully.")
    except requests.exceptions.Timeout:
        print("The request timed out.")
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        
if not os.path.exists(CIFAR_ROOT_PATH):
    with tarfile.open(CIFAR_TAR_PATH) as tar:
        tar.extractall(path=DATA_PATH)

In [ ]:
# utils for preprocessing data

Data = namedtuple('Data', 'x y')

def load_data(file_path):
    with open(file_path, 'rb') as file:
        dict = pickle.load(file, encoding='bytes')
    return dict[b'data'], dict[b'labels']

def load_cifar10(number_of_batches):
    assert 0 < number_of_batches < 6
    data = []
    labels = []
    test_data, test_labels = load_data(TEST_DATA_BATCH_PATH)
    for i in range(1, number_of_batches + 1):
        d, l = load_data(os.path.join(CIFAR_ROOT_PATH, f"data_batch_{i}"))
        data.append(d)
        labels.append(l)
    # in data rows are samples so stack them on the rows with vstack
    # in labels columns are samples so stack them on the columns with hstack
    return np.vstack(data), np.hstack(labels), np.array(test_data), np.array(test_labels)

def train_valid_split(data, labels, split_ratio):
    assert type(split_ratio) is list and len(split_ratio) == 2 
    x_train, x_valid, y_train, y_valid = train_test_split(data, labels, train_size=split_ratio[0], test_size=split_ratio[1], shuffle=False)
    return x_train, y_train, x_valid, y_valid
    
def transform_data(x_train, x_valid, x_test, y_train, y_valid, y_test):
    # normalize data
    mean = np.mean(x_train, axis=0)
    std = np.std(x_train, axis=0)
    x_train = (x_train - mean) / std
    x_valid = (x_valid - mean) / std
    x_test = (x_test - mean) / std
    assert np.allclose(np.mean(x_train, axis=0), np.zeros(x_train.shape[1]))
    assert np.allclose(np.std(x_train, axis=0), np.ones(x_train.shape[1]))
    
    # transform data to d x n to adhere to the gradient formulas
    x_train = x_train.T
    x_valid = x_valid.T
    x_test = x_test.T
    
    # transform labels to one-hot encoding
    y_train_one_hot = np.zeros((10, y_train.size))
    y_train_one_hot[y_train, np.arange(y_train.size)] = 1
    y_valid_one_hot = np.zeros((10, y_valid.size))
    y_valid_one_hot[y_valid, np.arange(y_valid.size)] = 1
    y_test_one_hot = np.zeros((10, y_test.size))
    y_test_one_hot[y_test, np.arange(y_test.size)] = 1
    
    return Data(x_train, y_train_one_hot), Data(x_valid, y_valid_one_hot), Data(x_test, y_test_one_hot)


In [ ]:
# Only load the first 20k data, first 10k is for training and other 10k is valid, test batch is a separate data with 10k
# data is wrapped in a Data namedtuple with field x for the data and y for the labels
x_data, y_data, x_test, y_test =  load_cifar10(2)
assert x_data.shape == (20000, 3072)
assert y_data.shape == (20000,)
assert x_test.shape == (10000, 3072)
assert y_test.shape == (10000,)

x_train, y_train, x_valid, y_valid = train_valid_split(x_data, y_data, [0.5, 0.5])
assert x_train.shape == (10000, 3072)
assert x_valid.shape == (10000, 3072)
assert y_train.shape == (10000,)
assert y_valid.shape == (10000,)

train, valid, test = transform_data(x_train, x_valid, x_test, y_train, y_valid, y_test)
assert train.x.shape == (3072, 10000)
assert valid.x.shape == (3072, 10000)
assert test.x.shape == (3072, 10000)
assert train.y.shape == (10, 10000)
assert valid.y.shape == (10, 10000)
assert test.y.shape == (10, 10000)

# Deriving the gradients for the NN

The computational graph we have for this network is illustrated below:

![](./assets/one_layer_nn_comp_graph.png)

where $J$ is the cost function, which takes the output from the final layer to compute the loss. Our functions are thus

$$J = l + \lambda r$$

$$r = \|w\|^2$$

$$l = -\log(y^T p)$$

$$p = \text{softmax}(s)$$

$$s = z + b$$

$$z = Wx$$

**Dimensions.** Note that 
* $y$ is a one-hot encoding, consisting of zeros except for one element that is 1, of size $c \times 1$ for a single data point and $c \times n$ for a dataset of size n. c stands for number of classes, in cifar10 that's 10 classes.
* $w$ is $c \times d$, where d is the number of features of the data.
* $x$ is $d \times 1$ for single data point and $d \times n$ for a batch of size n.
* $s$ is $c \times 1$ and $c \times n$.
* $p$ is $c \times 1$ and $c \times n$ for a batch.
* $l$ and $J$ both are scalars because they are loss functions.
* $r$ is a scalar, because it's the frobenius norm (l2-norm) of the matrix $w$.

At the core the process consists of just being comfortable with the chain rule, because that sets up the structure of the gradients, and being comfortable with matrix calculus is beneficial to actually compute the gradients, otherwise use the [matrix cookbook](https://www.math.uwaterloo.ca/~hwolkowi/matrixcookbook.pdf) if some of them are too tricky. Also knowing the dimensions of the variables and the intermediate results is crucial, both in coding and deriving the gradients.

We want to find the optimal parameters for the one linear layer neural network. This nn has $w$ and $b$ as model parameters, so these are the gradients we are looking for

$$\frac{\partial J}{\partial w} ,\, \frac{\partial J}{\partial b}$$

Following the computational graph, the backprop starts from the end and propagates to the start. Applying the chain rule on J w.r.t w we therefore get when we go backwards from J to z:

$$\frac{\partial J}{\partial w} = \textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}} \frac{\partial s}{\partial z} \frac{\partial z}{\partial w} + \frac{\partial J}{\partial r} \frac{\partial r}{\partial w}$$

similarily we get for J w.r.t b:
$$\frac{\partial J}{\partial b} = \textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}} \frac{\partial s}{\partial b}$$

I've highlighted the red part that occurs in both gradients. In the code we can be more efficient and store this as a variable to avoid recomputing it each time.

We start with one single data point and later extend it to a batch of data. Moreover, we use the [numerator layout](https://en.wikipedia.org/wiki/Matrix_calculus#Numerator-layout_notation) when computing the gradients. For clarification here's a simple example that should illustrate the idea of the numerator layout

$$
\frac{\partial f}{\partial x} =
\begin{bmatrix}
\frac{\partial f_{1}}{\partial x_{1}} \dots{} \frac{\partial f_{1}}{\partial x_{n}} \\
\vdots \\
\frac{\partial f_{m}}{\partial x_{1}} \dots{} \frac{\partial f_{m}}{\partial x_{n}}
\end{bmatrix}
$$

For row 1 we fix f and vary x. For row 2 we fix the next element in f, which is $f_{2}$ and vary x the same way. We do this all the way down to the last row m, which is we fix $f_m$ and vary x because there are only m outputs from the vector-valued function $f: \mathbb{R}^n \to \mathbb{R}^m$. This produces a $m \times n$ jacobian matrix that is the derivative of the vector-valued function $f$ w.r.t the vector $x$.
This pattern naturally applies to gradients and scalar functions with scalar inputs as well. So really it's a general method that works for all kinds of situations, as long as the derivative exists.


## Computing $\frac{\partial J}{\partial b}$
The gradient of J w.r.t b is the shortest so we start with it. Recall that we had 

$$\frac{\partial J}{\partial b} = \textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}} \frac{\partial s}{\partial b}$$

#### $\frac{\partial J}{\partial l}$
Computing one gradient at a time we have $\frac{\partial J}{\partial l} = \frac{\partial scalar}{\partial scalar} = scalar$ and given the function is $J = l + \lambda r$ we get

$$\frac{\partial J}{\partial l} = 1$$

#### $\frac{\partial l}{\partial p}$

For the next one remember that $\frac{\partial l}{\partial p}$ is of the form $\frac{\partial \text{scalar}}{\partial \text{vector}} = \text{vector}$, so we should get a gradient from this keeping in mind that we are using the numerator layout. The function is $l = -\log{y^T p}$ so we get

$$\frac{\partial l}{\partial p} = - \left(\frac{\partial l}{\partial p_1}, \frac{\partial l}{\partial p_2}, \dots, \frac{\partial l}{\partial p_c} \right) = - \left( \frac{y_1}{y^T p}, \dots, \frac{y_n}{y^T p} \right) = - \frac{y^T}{y^T p}$$

####  $\frac{\partial p}{\partial s}$

This one is not trivial, but applying the [quotient rule](https://en.wikipedia.org/wiki/Quotient_rule) for derivative will get you the answer, and with a smart conversion one can express the answer in matrix form. The function is $p = \text{softmax}(s) = \dfrac{\exp{s}}{1_c \exp{s}}$ where $1_c$ is a vector of size $c \times 1$ and s is the same size, which is the same as summing all the entries $\sum_j^c \exp{s_j}$. We get

$$\frac{\partial p_i}{\partial s_i} = \frac{\exp{s_i} (1_c \exp{s}) - \exp{s_i}\exp{s_i}}{(1_c \exp{s})^2} = \frac{\exp{s_i}}{1_c \exp{s}} \cdot \frac{1_c \exp{s} - \exp{s_i}}{1_c \exp{s}} = p_i(1-p_i), \ \ \  i=j$$

$$\frac{\partial p_i}{\partial s_j} = \frac{-\exp{s_i} \exp{s_j}}{(1_c \exp{s})^2} = -\frac{\exp{s_i}}{1_c \exp{s}} \cdot \frac{\exp{_js}}{1_c \exp{s}} = -(p_i p_j), \ \ \  i \neq j$$

$$
\iff 
\frac{\partial p}{\partial s} = 
\begin{cases}
p_i - p_i^2 \, ,i=j \\
-p_i p_j \, ,i \neq j
\end{cases}

\implies 
\frac{\partial p}{\partial s} = \text{diag}(p) - pp^T
$$

To arrive at the matrix form observe that $p_i$ is duplicated as a multiplication over all entries in the matrix as a negative factor $-p_i$, which suggests that we should use outer product $pp^T$ to get that form. Furthermore, at the diagonals we have a positive $p_i$, which suggests that we should take the $diag(p)$ and subtract it with the outer product of $pp^T$.

### $\frac{\partial s}{\partial b}$
We have the situation $\frac{\partial \text{vector}}{\partial \text{vector}} = \text{matrix}$ and observing that the function is $s = z + b$, the numerator layout will give

$$
\frac{\partial s}{\partial b} =
\begin{bmatrix}
\frac{\partial s_{1}}{\partial b_{1}} \dots{} \frac{\partial s_{1}}{\partial b_{c}} \\
\vdots \\
\frac{\partial s_{c}}{\partial b_{1}} \dots{} \frac{\partial s_{c}}{\partial b_{c}}
\end{bmatrix} =
\begin{bmatrix}
1 & 0 & \cdots & 0 \\
0 & 1 & \cdots & 0 \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & 1
\end{bmatrix} =
I_c
$$

where we can see that it's only on the diagonal that we get a non-zero result, in fact 1. This makes it an identity matrix of size $c \times c$.

### Putting $\frac{\partial J}{\partial b}$ together
Putting it together we get 

$$
\begin{align}
    \frac{\partial J}{\partial b} 
    &= \frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s} \frac{\partial s}{\partial b} \\
    &= 1 \cdot (-\frac{y^T}{y^T p}) \cdot (\text{diag}(p) - pp^T) \cdot I_c \\
    &= (-\frac{y^T}{y^T p}) \cdot (\text{diag}(p) - pp^T) \\
    &= (-\frac{1}{p_i})(\text{diag}(p) - pp^T)_i^T \\
    &= (-\frac{1}{p_i})(\text{diag}(p)_i - p_ip^T) \\
    &= (-\frac{1}{p_i})(p_i [0 \dots 1_i \dots 0] - p)^T \\
    &= -(y - p)^T
\end{align}
$$

The biggest insight comes from looking at $\text{diag}(p) - pp^T$ (I recommend to draw this) 

$$
\text{diag}(\mathbf{p}) - \mathbf{p} \mathbf{p}^T =
\begin{bmatrix}
p_1 - p_1^2 & -p_1 p_2 & \cdots & -p_1 p_c \\
-p_2 p_1 & p_2 - p_2^2 & \cdots & -p_2 p_c \\
\vdots & \vdots & \ddots & \vdots \\
-p_c p_1 & -p_c p_2 & \cdots & p_c - p_c^2
\end{bmatrix}
$$

and noticing that the one-hot encoding $y^T$ will choose the ith row of this matrix. You can further simplify that row by realizing that you can factor out $p_i$ from $\text{diag}(p)_i - p_ip^T$ and cancel it with $-\frac{1}{p_i}$, which leaves $-(y - p)^T$.

#

## Computing $\frac{\partial J}{\partial w}$

Recall we had the chain of gradients 

$$\frac{\partial J}{\partial w} = \textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}} \frac{\partial s}{\partial z} \frac{\partial z}{\partial w} + \textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w}}$$

### $\textcolor{orange}{\frac{\partial J}{\partial r} \frac{\partial r}{\partial w}}$

The functions are $J = l + \lambda r$ and $r = \|w\|^2$ and it's easy to see that we get 

$$\frac{\partial J}{\partial r} \frac{\partial r}{\partial w} = \lambda 2w$$

### $\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s} \frac{\partial s}{\partial z}}$

We earlier computed this to be $-(y-p)^T$, but for convenience sake, we will set it as $g^T = -(y-p)^T$. This has the size $1 \times c$.

### $\frac{\partial s}{\partial z}$

The function is $s = z + b$, so we can see that we get the exact same situation as $\frac{\partial s}{\partial b} = I_c$.

### $\frac{\partial z}{\partial w}$

The function is $z = wx$.
This is probably the hardest one, making the computations more complicated because of the introduction of [kronecker product](https://en.wikipedia.org/wiki/Kronecker_product) that is defined as

$$
\mathbf{A} \otimes \mathbf{B} =
\begin{bmatrix}
a_{11} \mathbf{B} & \cdots & a_{1n} \mathbf{B} \\
\vdots & \ddots & \vdots \\
a_{m1} \mathbf{B} & \cdots & a_{mn} \mathbf{B}
\end{bmatrix}
$$

where A and B are matrices.
We notice that we cannot solve 

$$\frac{\partial z}{\partial w}$$

in the traditional sense, since there isn't a standardized procedure for gradient of vector w.r.t a matrix. There are however different ways to solve it, one way in which the course did was to vectorize the matrix w by $\operatorname{vec}(w)$, write $z = wx = I_c wx$ and then use the following [relation](https://en.wikipedia.org/wiki/Vectorization_(mathematics)#Compatibility_with_Kronecker_products) $\operatorname{vec}(ABC) = (C^T \otimes A)\operatorname{vec}(B)$ to take the derivative of w.r.t $vec(B)$, which just removes it in the expression. Basically we get 

$$\frac{\partial \operatorname{vec}(z)}{\partial vec(w)} = \frac{\partial \operatorname{vec}(I_c wx)}{\partial vec(w)} = \frac{\partial (x^T \otimes I_c) \operatorname{vec}(w)}{\partial vec(w)} = (x^T \otimes I_c)$$

Notice that this has the size of $c \times cd$.

## Putting together $\frac{\partial J}{\partial w} = \textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s}} \frac{\partial s}{\partial z} \frac{\partial z}{\partial w} + \frac{\partial J}{\partial r} \frac{\partial r}{\partial w}$

Recall that $\textcolor{red}{\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s} \frac{\partial s}{\partial z}} = g^T$ is of size $1 \times c$ and $\frac{\partial z}{\partial w} = (x^T \otimes I_c)$ is size $c \times cd$. Multiplying them together we get 

$$
\begin{align}
\frac{\partial J}{\partial l} \frac{\partial l}{\partial p} \frac{\partial p}{\partial s} \frac{\partial s}{\partial z} \frac{\partial z}{\partial w} &= g^T(x^T \otimes I_c) \\
&= [g^Tx_1, g^Tx_2, \dots, g^Tx_d] \\
& \implies 
\begin{bmatrix}
g^Tx_1 \\
g^Tx_2 \\
\vdots \\
g^Tx_d
\end{bmatrix} \\
&= xg^T
\end{align}
$$

where we have converted the $1 \times cd$ matrix $[g^Tx_1, g^Tx_2, \dots, g^Tx_d]$ to size $d \times c$. We will need to transpose this from $(xg^T)^T = gx^T$ to make it size $c \times d$ to be able to add with $\frac{\partial J}{\partial r} \frac{\partial r}{\partial w}$ that is of size $c \times d$. The idea of simplifying the multiplication on the first line is that the identity matrix in the kronecker product will simplify alot, because of the many zero entries in the matrix. Finally we get the gradient

$$\frac{\partial J}{\partial w} = gx^T + 2\lambda w$$

## Extending to batches of data

Remember that the gradients computed

$$\frac{\partial J}{\partial w} = gx^T + 2\lambda w$$

$$\frac{\partial J}{\partial b} = -(y-p)^T = g^T$$

were for a single data point. However, we can easily extend this to a batch of data of arbitrary size. Assume $x_b$ is of size $d \times n$ and one-hot encoding labels $y_b$ is size $c \times n$, where n is the size of the data. Then in the forward pass we have

$$p_b = \operatorname{softmax}(wx_b + b 1^T_n)$$

where 
* $p_b$ is $c \times n$
* $w$ is $c \times d$
* $b 1^T_n$ is $c \times n$ and $1^T_n$ is $1 \times n$ which is used to broadcast of the bias vector $b$ where it's duplicated for each data point. 

In the backward pass we have a natural extension of the gradients we already computed

$$g_b = -(y_b - p_b)$$

$$\frac{\partial J}{\partial w} = \frac{1}{n} g_b x_b^T + 2\lambda w$$

$$\frac{\partial J}{\partial b} = \frac{1}{n} g_b 1_n$$

What's new here is that we need to take the average of the gradient in both, because we are doing it over batches. The bias gradient needs to sum all cols (features) and take the average of them, whereas the weight gradient is sort of doing the same, multiplying together the corresponding features from the $g_b$ matrix and the data $x_b^T$ and then adding regularization and taking the average of the features. This becomes a weight gradient of size $c \times n$ that can now update the weight matrix $w$ that is also of size $c \times n$ naturally (otherwise wouldn't be able to update).

-----------------------------------------------


# Implementing the nn

Back to the code. We now have the gradient formulas, so just implement it. We will need to evaluate the gradients with centered difference method to check for correctness. According to the lab, the relative error of magnitude should not be higher than $1\exp{-6}$.